<a href="https://colab.research.google.com/github/swapnabanoth/001_BDA_Assignment/blob/main/BDA_Assignment_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1 Build a Classification Model with Spark with a dataset of your choice

from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Step 1: Create Spark session
spark = SparkSession.builder.appName("IrisClassification").getOrCreate()

# Step 2: Load dataset
# You can replace this with your own CSV path
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# Downloading data and saving locally (if needed)
import pandas as pd
iris_df = pd.read_csv(data_url, names=columns)
iris_df.to_csv("iris.csv", index=False)

# Load into Spark
df = spark.read.csv("iris.csv", header=True, inferSchema=True)

# Step 3: Preprocess data
# Let's convert this into a binary classification problem: 'Iris-setosa' vs others
df = df.withColumn("label", (df["species"] == "Iris-setosa").cast("integer"))

# Features into a single vector
feature_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

# Step 4: Split dataset
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Step 5: Build and train model
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_data)

# Step 6: Make predictions
predictions = model.transform(test_data)

# Step 7: Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

# Optional: Show some predictions
predictions.select("features", "label", "prediction").show(5)

# Stop Spark session
spark.stop()


# 2 Build a Clustering Model with Spark with a dataset of your choice

from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Step 1: Spark Session
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# Step 2: Load dataset
# MovieLens 100k dataset: userId, movieId, rating, timestamp
import pandas as pd
import urllib.request

# Download MovieLens 100k
ml_url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
urllib.request.urlretrieve(ml_url, "ml-latest-small.zip")

import zipfile
with zipfile.ZipFile("ml-latest-small.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

# Read ratings into Spark
ratings_df = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
ratings_df = ratings_df.select("userId", "movieId", "rating")

# Step 3: Train-test split
train_data, test_data = ratings_df.randomSplit([0.8, 0.2], seed=42)

# Step 4: ALS model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"  # drop NaN predictions
)
model = als.fit(train_data)

# Step 5: Predictions
predictions = model.transform(test_data)

# Step 6: Evaluate the model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"\nRoot-mean-square error (RMSE): {rmse:.4f}")

# Step 7: Recommend Top 5 Movies for Each User
user_recs = model.recommendForAllUsers(5)
print("\nTop 5 movie recommendations for sample users:")
user_recs.show(5, truncate=False)

# Step 8: Stop Spark session
spark.stop()


# 3 Build a Recommendation Engine with Spark with a dataset of your choice
# Step 1: Setup Spark
# ============================
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

# Start Spark session
spark = SparkSession.builder.appName("MovieRecommendationALS").getOrCreate()

# ============================
# Step 2: Load Dataset
# ============================
ratings_path = "/content/ml-latest-small/ratings.csv"  # <- Change path if needed

df = spark.read.csv(ratings_path, header=True, inferSchema=True)
df = df.select("userId", "movieId", "rating")
df.show(5)

# ============================
# Step 3: Train ALS Model
# ============================
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    maxIter=10,
    regParam=0.1,
    rank=10,
    nonnegative=True,
    coldStartStrategy="drop"
)

model = als.fit(df)

# ============================
# Step 4: Make Predictions
# ============================
predictions = model.transform(df)

# ============================
# Step 5: Evaluate the Model
# ============================
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"\n Root Mean Squared Error (RMSE): {rmse:.2f}")

# ============================
# Step 6: Generate Recommendations
# ============================
userRecs = model.recommendForAllUsers(5)
itemRecs = model.recommendForAllItems(5)

print("\n Top-5 movie recommendations for users:")
userRecs.select("userId", "recommendations").show(5, truncate=False)

# ============================
# Step 7: Stop Spark
# ============================
spark.stop()
